# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [25]:
# import libraries
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [26]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('cleaned_data', engine)

# Clean dataframe remove na values
df.dropna(axis='index',inplace=True)

df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# Preparing data
X = df.message.values
t = df.drop(columns=['id', 'message', 'original', 'genre']).astype(int)

X = df['message']
Y = t

print(X.shape)
print(Y.shape)

(10170,)
(10170, 36)


### 2. Write a tokenization function to process your text data

In [28]:
import re
import nltk
import ssl

# running nltk download gave ssl error.
# Resolution reference: https://github.com/gunthercox/ChatterBot/issues/930#issuecomment-322111087
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context


nltk.download(['punkt', 'wordnet', 'stopwords'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)
    
    stop_word = stopwords.words("english")
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    
    lemmed = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_word]
    
    return lemmed

[nltk_data] Downloading package punkt to /Users/vasthav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/vasthav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vasthav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [29]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 10, n_jobs=-1)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [31]:
# predict on test data
predicted = pipeline.predict(X_test)
predicted

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [32]:
# Finding the Accuracy
accuracy = (predicted == y_test).mean()
print(f'accuracy = {accuracy.mean()} \n')

accuracy = 0.955116441648097 



In [33]:
# references:
# https://stackoverflow.com/questions/38697982/python-scikit-learn-multi-class-multi-label-performance-metrics
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html?highlight=report#sklearn.metrics.classification_report

from sklearn.metrics import classification_report

y_pred_pd = pd.DataFrame(predicted, columns = y_test.columns)
for column in y_test.columns:
    print(f'Class: {column}\n')
    print(classification_report(y_test[column],y_pred_pd[column]))

Class: related

              precision    recall  f1-score   support

           0       0.68      0.64      0.66       825
           1       0.83      0.86      0.84      1718

    accuracy                           0.79      2543
   macro avg       0.76      0.75      0.75      2543
weighted avg       0.78      0.79      0.78      2543

Class: request

              precision    recall  f1-score   support

           0       0.82      0.90      0.86      1638
           1       0.78      0.65      0.71       905

    accuracy                           0.81      2543
   macro avg       0.80      0.77      0.78      2543
weighted avg       0.81      0.81      0.81      2543

Class: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2542
           1       0.00      0.00      0.00         1

    accuracy                           1.00      2543
   macro avg       0.50      0.50      0.50      2543
weighted avg       1.00     

/Users/vasthav/Udacity/udacityenv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      0.99      0.97      2264
           1       0.86      0.53      0.65       279

    accuracy                           0.94      2543
   macro avg       0.90      0.76      0.81      2543
weighted avg       0.93      0.94      0.93      2543

Class: clothing

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2514
           1       0.00      0.00      0.00        29

    accuracy                           0.99      2543
   macro avg       0.49      0.50      0.50      2543
weighted avg       0.98      0.99      0.98      2543

Class: money

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2512
           1       0.00      0.00      0.00        31

    accuracy                           0.99      2543
   macro avg       0.49      0.50      0.50      2543
weighted avg       0.98      0.99      0.98

### 6. Improve your model
Use grid search to find better parameters. 

In [34]:
# Find supported parameters keys.
list(pipeline.get_params().keys())

['memory',
 'steps',
 'verbose',
 'vect',
 'tfidf',
 'clf',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram_range',
 'vect__preprocessor',
 'vect__stop_words',
 'vect__strip_accents',
 'vect__token_pattern',
 'vect__tokenizer',
 'vect__vocabulary',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf',
 'clf__estimator__bootstrap',
 'clf__estimator__ccp_alpha',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__max_samples',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_impurity_split',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__n_estimators',
 'clf__estimator__n_jobs',
 'c

In [35]:
# parameters = 

# cv = 
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
    'vect__max_features': (None, 5000), 
    'clf__estimator__n_estimators': [10, 20] 
}

# parametes = {
#     'clf__estimator__n_estimators': [10],
#     'vect__ngram_range': [(1, 1)]
# }

# cv = GridSearchCV(estimator=pipeline, param_grid=parameters)

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, scoring='f1_macro', cv=None, n_jobs=-1,verbose=10)
cv.fit(X_train, y_train)


Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   59.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 18.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 22.4min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 51

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [36]:
print("done!")

print(cv.best_params_)
print(cv.best_score_)
print(cv.param_grid)

done!
{'clf__estimator__n_estimators': 20, 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__max_features': 5000, 'vect__ngram_range': (1, 2)}
0.2504474031546078
{'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_df': (0.5, 0.75, 1.0), 'vect__max_features': (None, 5000), 'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20]}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [37]:
y_predicted_cv = cv.predict(X_test)

# Finding the Accuracy
accuracy = (predicted == y_test).mean()
print(f'Accuracy = {round(accuracy.mean() * 100,2)}% \n')

Accuracy = 95.51% 



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [38]:
import pickle
pickle.dump(cv, open('../models/disaster_response_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.